In [2]:
import numpy as np
import os
from tqdm import tqdm

from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.dummy import DummyClassifier
from scipy.stats import t

In [3]:
# for 5x2 cv paired t test, calculation of s² for 1 replication
def si2(pA,pB): # pA = [pA1,pA2] and pB = [pB1,pB2]
    p1 = pA[0] - pB[0]
    p2 = pA[1] - pB[1]
    pbar = (p1 + p2)/2
    s2 = (p1 - pbar)**2 + (p2 - pbar)**2
    return(s2)

In [5]:
# labels importation

drawings_names = os.listdir('img/')
seasons = ['Autumn','Winter','Spring','Summer']
i = 0
labels = list()

for img in tqdm(drawings_names):
    if not img.startswith('.'):
        for season in seasons:
            if season in img:
                labels.append(str(season))
        i = i + 1

100%|██████████| 1300/1300 [00:00<00:00, 349749.53it/s]


## Compare RGB with random models

In [5]:
repertory_name = 'features/PCA_table/'
repertory = os.listdir(repertory_name)

for layer in repertory:
    if not 'checkpoint' in layer:
        X_pca = np.load(repertory_name + '/' + str(layer))
        X_pca = X_pca['arr_0']

        scores_SVC = []
        scores_dummy = []
        s2 = []
        for i in range(5): # to repeat 5 times
            # 2-fold cv on SVC and dummy classifiers
            clf = SVC()
            dummy_clf = DummyClassifier(strategy = 'most_frequent')
            skf = StratifiedKFold(2, shuffle = True, random_state = i)
            scores_dummy.append(cross_val_score(dummy_clf, X_pca, labels, cv = skf))
            scores_SVC.append(cross_val_score(clf, X_pca, labels, cv = skf))

            # calculate si² for i =1...5
            s2.append(si2(scores_SVC[i], scores_dummy[i]))

        # calculate the t statistic and conclude : if p < 0.05 : reject H0, meaning classifiers don't have equal performance

        numerator = scores_SVC[0][0] - scores_dummy[0][0]
        denominator = np.sqrt((1/5)*np.sum(s2))
        t_statistic = numerator/denominator
        pvalue = t.sf(abs(t_statistic), df=5)*2
        print('Layer : ' + str(layer.split('_')[2].split('.')[0]) + ' | p-value : ' + str(pvalue) + ' | t-value : ' + str(t_statistic))

Layer : 3 | p-value : 0.018960248968718982 | t-value : 3.4140359124885507
Layer : 5 | p-value : 0.008933086124580745 | t-value : 4.147343007422349
Layer : 7 | p-value : 0.0042078801648281474 | t-value : 4.97107229430879
Layer : fc2 | p-value : 0.006648000586002324 | t-value : 4.458776100346009
Layer : 9 | p-value : 0.010993482147370807 | t-value : 3.936945502457753
Layer : 10 | p-value : 0.004977615772805138 | t-value : 4.778413125096315
Layer : 0 | p-value : 0.006152093025957708 | t-value : 4.542943789728608
Layer : 8 | p-value : 0.00048334596159133275 | t-value : 8.033481501505136
Layer : 0a | p-value : 0.004895573752320558 | t-value : 4.797234573563938
Layer : 4 | p-value : 0.009090917157889629 | t-value : 4.129329068132486
Layer : 6 | p-value : 0.0019691004868953375 | t-value : 5.914040070180881
Layer : 14 | p-value : 0.000117930508206652 | t-value : 10.8024279088095
Layer : 11 | p-value : 0.027470000164592296 | t-value : 3.079988270219044
Layer : fc1 | p-value : 0.0174937457252545

## Compare grayscale with random models

In [6]:
repertory_name = 'features_BW/PCA_table_BW/'
repertory = os.listdir(repertory_name)

for layer in repertory:
    if not 'checkpoint' in layer:
        X_pca = np.load(repertory_name + '/' + str(layer))
        X_pca = X_pca['arr_0']

        scores_SVC = []
        scores_dummy = []
        s2 = []
        for i in range(5): # to repeat 5 times
            # 2-fold cv on SVC and dummy classifiers
            clf = SVC()
            dummy_clf = DummyClassifier(strategy = 'most_frequent')
            skf = StratifiedKFold(2, shuffle = True, random_state = i)
            scores_dummy.append(cross_val_score(dummy_clf, X_pca, labels, cv = skf))
            scores_SVC.append(cross_val_score(clf, X_pca, labels, cv = skf))

            # calculate si² for i =1...5
            s2.append(si2(scores_SVC[i], scores_dummy[i]))

        # calculate the t statistic and conclude : if p < 0.05 : reject H0, meaning classifiers don't have equal performance

        numerator = scores_SVC[0][0] - scores_dummy[0][0]
        denominator = np.sqrt((1/5)*np.sum(s2))
        t_statistic = numerator/denominator
        pvalue = t.sf(abs(t_statistic), df=5)*2
        sign = ''
        if pvalue<0.05:
            sign = ' *'
        print('Layer : ' + str(layer.split('_')[3].split('.')[0]) + ' | p-value : ' + str(pvalue) + str(sign) + ' | t-value : ' + str(t_statistic))

Layer : 1 | p-value : 0.06537231015137396 | t-value : 2.35230958409157
Layer : 4 | p-value : 0.03483926771536033 * | t-value : 2.873806092299536
Layer : 15 | p-value : 0.03474627531227645 * | t-value : 2.8760934190440723
Layer : fc2 | p-value : 6.818100917723113e-06 * | t-value : 19.33997738801598
Layer : 6 | p-value : 0.004105786156890457 * | t-value : 4.999695575684992
Layer : 9 | p-value : 0.0002977863307578082 * | t-value : 8.901850931172238
Layer : 11 | p-value : 0.05489026704562209 | t-value : 2.49402011162011
Layer : 14 | p-value : 0.0006075800478197828 * | t-value : 7.649990807125729
Layer : 13 | p-value : 0.0004084925589397374 * | t-value : 8.326213451166979
Layer : 0 | p-value : 0.02658909326978062 * | t-value : 3.1087264930397955
Layer : 10 | p-value : 0.003497063230470922 * | t-value : 5.189647871707591
Layer : 8 | p-value : 0.0024131676875803415 * | t-value : 5.6492411143755366
Layer : 2 | p-value : 0.10200929057182935 | t-value : 1.999463923315085
Layer : 5 | p-value : 0.

In [ ]:
### significativité pour les layers :
# 0, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, FC1, FC2
# pas significatif pour les layers 1, 2, 11 